In [ ]:
import numpy as np
import qubex as qx

from qubex.simulator import (
    Control,
    Coupling,
    QuantumSimulator,
    QuantumSystem,
    SimulationResult,
    Transmon,
)

qx.pulse.set_sampling_period(0.1)

In [ ]:
# units: GHz, ns
transmon_dimension = 3

control_qubit_label = "Q04"
control_qubit_frequency = 7157.231e-3
control_anharmonicity = -393.715e-3

target_qubit_label = "Q01"
target_qubit_frequency = 8032.295e-3
target_anharmonicity = -487.412e-3

relaxation_rate = 0.05e-3
dephasing_rate = 0.05e-3

coupling_strength = 5e-3
frequency_diff = control_qubit_frequency - target_qubit_frequency

pi_duration = 24

cr_amplitude_max = 0.75 * abs(frequency_diff)
cr_phase_offset = 0.25 * np.pi
cr_ramptime = 16

xt_ratio = 0.01
xt_phase_offest = 0.5 * np.pi

rotary_multiple = 17
duration_unit = 16

In [ ]:
control_qubit = Transmon(
    label=control_qubit_label,
    dimension=transmon_dimension,
    frequency=control_qubit_frequency,
    anharmonicity=control_anharmonicity,
    relaxation_rate=relaxation_rate,
    dephasing_rate=dephasing_rate,
)

target_qubit = Transmon(
    label=target_qubit_label,
    dimension=transmon_dimension,
    frequency=target_qubit_frequency,
    anharmonicity=target_anharmonicity,
    relaxation_rate=relaxation_rate,
    dephasing_rate=dephasing_rate,
)

couplings = [
    Coupling(
        pair=(control_qubit_label, target_qubit_label),
        strength=coupling_strength,
    ),
]

system = QuantumSystem(
    objects=[control_qubit, target_qubit],
    couplings=couplings,
)

simulator = QuantumSimulator(system)

In [ ]:
pi_pulse = qx.pulse.Drag(
    duration=pi_duration,
    amplitude=1,
    beta=-0.5 / (2 * np.pi * control_qubit.anharmonicity),
)
norm_factor = np.pi / float(np.sum(np.abs(pi_pulse.values) * pi_pulse.SAMPLING_PERIOD))
pi_pulse = pi_pulse.scaled(norm_factor)
pi_pulse.plot()

result = simulator.mesolve(
    controls=[
        Control(
            target=control_qubit_label,
            frequency=control_qubit_frequency,
            waveform=pi_pulse,
        )
    ],

    initial_state={
        control_qubit_label: "0",
    },
)
result.display_bloch_sphere(control_qubit_label)
result.show_last_population(control_qubit_label)

In [ ]:
def cr_sequence(
    duration: float,
    cr_amplitude: float,
    cr_phase: float,
    cancel_amplitude: float,
    cancel_phase: float,
    echo: bool,
):
    cr_beta = -1 / (2 * np.pi * frequency_diff)
    cr_waveform = qx.pulse.FlatTop(
        duration=duration,
        amplitude=2 * np.pi * cr_amplitude,
        tau=cr_ramptime,
        phase=cr_phase + cr_phase_offset
    )
    cancel_waveform = qx.pulse.FlatTop(
        duration=duration,
        amplitude=2 * np.pi * cancel_amplitude,
        tau=cr_ramptime,
        phase=cancel_phase
    )
    crosstalk_waveform = qx.pulse.FlatTop(
        duration=duration,
        amplitude=2 * np.pi * cr_amplitude * xt_ratio,
        tau=cr_ramptime,
        phase=xt_phase_offest
    )
    targets = [
        qx.PulseChannel(
            label="Control",
            frequency=control_qubit_frequency,
            target=control_qubit_label,
        ),
        qx.PulseChannel(
            label="CR",
            frequency=target_qubit_frequency,
            target=control_qubit_label,
        ),
        qx.PulseChannel(
            label="Crosstalk",
            frequency=target_qubit_frequency,
            target=target_qubit_label,
        ),
        qx.PulseChannel(
            label="Target",
            frequency=target_qubit_frequency,
            target=target_qubit_label,
        ),
    ]
    with qx.PulseSchedule(targets) as cr:
        cr.add("CR", cr_waveform)
        cr.add("Crosstalk", crosstalk_waveform)
        cr.add("Target", cancel_waveform)

    if not echo:
        sched = cr
    else:
        with qx.PulseSchedule(targets) as ecr:
            ecr.call(cr)
            ecr.barrier()
            ecr.add("Control", pi_pulse)
            ecr.barrier()
            ecr.call(cr.scaled(-1))
            ecr.barrier()
            ecr.add("Control", pi_pulse)
        sched = ecr
    return sched

In [ ]:
def simulate_cr(
    cr_amplitude: float,
    cr_phase: float,
    cancel_amplitude: float,
    cancel_phase: float,
    duration: float,
    echo: bool,
    control_state: str,
    n_samples: int = 256,
    plot: bool = False,
) -> SimulationResult:
    controls = cr_sequence(
        cr_amplitude=cr_amplitude,
        cr_phase=cr_phase,
        cancel_amplitude=cancel_amplitude,
        cancel_phase=cancel_phase,
        duration=duration,
        echo=echo,
    )
    initial_state = system.state(
        {
            control_qubit.label: control_state,
            target_qubit.label: "0",
        },
    )
    result = simulator.mesolve(
        controls=controls,
        initial_state=initial_state,
        n_samples=n_samples,
    )
    if plot:
        result.plot_bloch_vectors(control_qubit_label)
        result.plot_bloch_vectors(target_qubit_label)
        result.display_bloch_sphere(control_qubit_label)
        result.display_bloch_sphere(target_qubit_label)
    return result

In [ ]:
result_0 = simulate_cr(
    cr_amplitude=cr_amplitude_max,
    cr_phase=0.0,
    cancel_amplitude=0.0,
    cancel_phase=0.0,
    duration=100,
    echo=False,
    control_state="0",
    plot=True,
)

In [ ]:
result_1 = simulate_cr(
    cr_amplitude=cr_amplitude_max,
    cr_phase=0.0,
    cancel_amplitude=0.0,
    cancel_phase=0.0,
    duration=100,
    echo=False,
    control_state="1",
    plot=True,
)

In [ ]:
def hamiltonian_tomography(
    cr_amplitude: float,
    cr_phase: float,
    cancel_amplitude: float,
    cancel_phase: float,
    duration: float = 1000,
    n_samples: int = 100,
    plot: bool = True,
) -> dict:
    result_0 = simulate_cr(
        cr_amplitude=cr_amplitude,
        cr_phase=cr_phase,
        cancel_amplitude=cancel_amplitude,
        cancel_phase=cancel_phase,
        duration=duration,
        echo=False,
        control_state="0",
        n_samples=n_samples,
        plot=plot,
    )
    result_1 = simulate_cr(
        cr_amplitude=cr_amplitude,
        cr_phase=cr_phase,
        cancel_amplitude=cancel_amplitude,
        cancel_phase=cancel_phase,
        duration=duration,
        echo=False,
        control_state="1",
        n_samples=n_samples,
        plot=plot,
    )

    times = result_0.times
    vectors_0 = result_0.get_bloch_vectors(target_qubit_label)
    vectors_1 = result_1.get_bloch_vectors(target_qubit_label)

    indices = (times >= cr_ramptime) & (times < times[-1] - cr_ramptime)
    times_ = times[indices] - cr_ramptime * 0.5
    vectors_0_ = vectors_0[indices]
    vectors_1_ = vectors_1[indices]

    fit_0 = qx.fit.fit_rotation(
        times_,
        vectors_0_,
        plot=False,
    )
    fit_1 = qx.fit.fit_rotation(
        times_,
        vectors_1_,
        plot=False,
    )
    Omega_0 = fit_0["Omega"]
    Omega_1 = fit_1["Omega"]
    Omega = np.concatenate(
        [
            0.5 * (Omega_0 + Omega_1),
            0.5 * (Omega_0 - Omega_1),
        ]
    )
    coeffs = dict(
        zip(
            ["IX", "IY", "IZ", "ZX", "ZY", "ZZ"],
            Omega / (2 * np.pi),
        )
    )

    print("Rotation coefficients:")
    for key, value in coeffs.items():
        print(f"  {key}: {value * 1e3:+.3f} MHz")

    xt_rotation = coeffs["IX"] + 1j * coeffs["IY"]
    print()
    print("XT (crosstalk) rotation:")
    print(f"  rate  : {np.abs(xt_rotation) * 1e3:.3f} MHz")
    print(f"  phase : {np.angle(xt_rotation):.3f} rad")

    cr_rotation = coeffs["ZX"] + 1j * coeffs["ZY"]
    print()
    print("CR (cross-resonance) rotation:")
    print(f"  rate  : {np.abs(cr_rotation) * 1e3:.3f} MHz")
    print(f"  phase : {np.angle(cr_rotation):.3f} rad")

    zx90_duration = 1 / (4 * np.abs(cr_rotation))
    print()
    print("Estimated ZX90 gate:")
    print(f"  drive    : {cr_amplitude * 1e3:.1f} MHz")
    print(f"  duration : {zx90_duration:.1f} ns")
    print()

    return {
        "Omega": Omega,
        "coeffs": coeffs,
        "xt_rotation": xt_rotation,
        "cr_rotation": cr_rotation,
        "zx90_duration": zx90_duration,
    }

In [ ]:
tomography_1 = hamiltonian_tomography(
    cr_amplitude=cr_amplitude_max,
    cr_phase=0.0,
    cancel_amplitude=0.0,
    cancel_phase=0.0,
    plot=True,
)

tomography_1

In [ ]:
cr_phase_calib = -np.angle(tomography_1["cr_rotation"])
cr_phase_calib

In [ ]:
tomography_2 = hamiltonian_tomography(
    cr_amplitude=cr_amplitude_max,
    cr_phase=cr_phase_calib,
    cancel_amplitude=0.0,
    cancel_phase=0.0,
    plot=True,
)

tomography_2

In [ ]:
cancel_pulse_max = -tomography_2["xt_rotation"]
cancel_amplitude_max = np.abs(cancel_pulse_max)
cancel_phase_max = np.angle(cancel_pulse_max)

cancel_amplitude_max, cancel_phase_max

In [ ]:
tomography_3 = hamiltonian_tomography(
    cr_amplitude=cr_amplitude_max,
    cr_phase=cr_phase_calib,
    cancel_amplitude=cancel_amplitude_max,
    cancel_phase=cancel_phase_max,
    plot=True,
)

tomography_3

In [ ]:
zx_rate = tomography_3["coeffs"]["ZX"]
zx90_duration = tomography_3["zx90_duration"]
zx_rate, zx90_duration

In [ ]:
cr_duration = ((zx90_duration / 2 + cr_ramptime) // duration_unit + 1) * duration_unit
cr_duration

In [ ]:
cr_sequence(
    duration=cr_duration,
    cr_amplitude=cr_amplitude_max,
    cr_phase=cr_phase_calib,
    cancel_amplitude=cancel_amplitude_max,
    cancel_phase=cancel_phase_max,
    echo=True,
).plot(
    title="Echoed Cross Resonance Sequence",
    width=800,
    divide_by_two_pi=True,
)

In [ ]:
def measure(cr_amplitude: float):
    ratio = cr_amplitude / cr_amplitude_max
    cancel_pulse = (cancel_pulse_max + zx_rate * rotary_multiple) * ratio
    result = simulate_cr(
        cr_amplitude=cr_amplitude,
        cr_phase=cr_phase_calib,
        cancel_amplitude=np.abs(cancel_pulse),
        cancel_phase=np.angle(cancel_pulse),
        duration=cr_duration,
        echo=True,
        control_state="0",
        n_samples=2,
        plot=False,
    )
    state = result.get_bloch_vectors(target_qubit.label)[-1]
    return state[2]

In [ ]:
amplitude_range = np.linspace(cr_amplitude_max * 0.8, cr_amplitude_max * 1.2, 20)

z_values = []
for amplitude in amplitude_range:
    result = measure(cr_amplitude=amplitude)
    z_values.append(result)

z_values = np.array(z_values)

qx.viz.plot(x=amplitude_range, y=z_values)

In [ ]:
fit_result = qx.fit.fit_polynomial(x=amplitude_range, y=z_values, degree=3)

cr_amplitude_calib = fit_result["root"]
cr_amplitude_calib

In [ ]:
ratio = cr_amplitude_calib / cr_amplitude_max
cancel_pulse_calib = (cancel_pulse_max + zx_rate * rotary_multiple) * ratio
cancel_amplitude_calib = np.abs(cancel_pulse_calib)
cancel_phase_calib = np.angle(cancel_pulse_calib)

cancel_amplitude_calib, cancel_phase_calib

In [ ]:
measure(cr_amplitude=cr_amplitude_calib)

In [ ]:
result_ecr_0 = simulate_cr(
    cr_amplitude=cr_amplitude_calib,
    cr_phase=cr_phase_calib,
    cancel_amplitude=cancel_amplitude_calib,
    cancel_phase=cancel_phase_calib,
    duration=cr_duration,
    control_state="0",
    echo=True,
    plot=True,
)

In [ ]:
result_ecr_1 = simulate_cr(
    cr_amplitude=cr_amplitude_calib,
    cr_phase=cr_phase_calib,
    cancel_amplitude=cancel_amplitude_calib,
    cancel_phase=cancel_phase_calib,
    duration=cr_duration,
    control_state="1",
    echo=True,
    plot=True,
)

In [ ]:
print("CR parameters:")
print(f"  duration:         {cr_duration} ns")
print(f"  ramptime:         {cr_ramptime} ns")
print(f"  cr_amplitude:     {cr_amplitude_calib * 1e3:.1f} MHz")
print(f"  cr_phase:         {cr_phase_calib:.3f} rad")
print(f"  cancel_amplitude: {cancel_amplitude_calib * 1e3:.1f} MHz")
print(f"  cancel_phase:     {cancel_phase_calib:.3f} rad")